In [1]:
import os
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from tensorflow.keras.utils import to_categorical

import numpy as np
import pickle
from torch.utils.data import DataLoader, Dataset

from models.GACNet.modules import GACNet, get_loss

import torch

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [3]:
with open('/home/songhao.jiang/enzyme/mesh/data/small_X.npy', 'rb') as handle:
    X = np.load(handle)
with open('/home/songhao.jiang/enzyme/mesh/data/small_y.npy', 'rb') as handle:
    y = np.load(handle)

In [4]:
normalize = True
for idx in range(len(X)):
    cur_x = X[idx]
    xyz = cur_x[:,:3]
    norm = cur_x[:,3:]

    centroid = np.mean(xyz, axis=0)
    xyz -= centroid
    if normalize:
        m = np.max(np.sqrt(np.sum(xyz**2, axis=1)))
        xyz = xyz / m

    points = np.concatenate([xyz, norm], 1)
    X[idx] = points

input_data = X
#target_data = to_categorical(y)
target_data = y
input_data.shape, target_data.shape

X_train, X_test, y_train, y_test = train_test_split(input_data, target_data, test_size=.1, random_state=0)
X_train.shape, y_train.shape


((5925, 25000, 6), (5925,))

In [5]:
class PC_dataset(Dataset):

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        pc = self.x[idx]
        label = self.y[idx]

        return pc, label

train_dataset = PC_dataset(X_train, y_train)
test_dataset = PC_dataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [28]:
NUM_CLASSES = 6

classifier = GACNet(NUM_CLASSES).cuda()
#criterion = get_loss()

criterion = get_loss().cuda()

optimizer = torch.optim.Adam(
    classifier.parameters(),
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-07,
    weight_decay=0
)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1:
        torch.nn.init.xavier_normal_(m.weight.data)
        torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find('Linear') != -1:
        torch.nn.init.xavier_normal_(m.weight.data)
        torch.nn.init.constant_(m.bias.data, 0.0)

def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace=True
classifier = classifier.apply(inplace_relu)
classifier = classifier.apply(weights_init)

In [29]:
for epoch in range(100):
    num_batches = len(train_loader)
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    classifier = classifier.train()
    mean_correct = []
    total_loss = 0
    for i, (points, target) in tqdm(enumerate(train_loader), total=len(train_loader), smoothing=0.9):
        optimizer.zero_grad()

        points = points.data.numpy()
        points = torch.Tensor(points)
        points = points.transpose(2, 1)
        norm = points[:, 3:, :]
        xyz = points[:, :3, :]
        
        xyz, norm, target = xyz.float().cuda(), norm.float().cuda(), target.long().cuda()
        #print(xyz.shape, norm.shape)
        pred = classifier(xyz, norm)
        loss = criterion(pred, target.long())


        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.long().data).cpu().sum()
        mean_correct.append(correct.item() / float(points.size()[0]))
        loss.backward()
        optimizer.step()
        total_loss += loss.cpu().detach().data.item()
    train_instance_acc = np.mean(mean_correct)
    print("Epoch: {}".format(epoch+1))
    print('Train Instance Accuracy: %f' % train_instance_acc)
    print('Train Loss: %f' % total_loss)

  0%|          | 0/2962 [00:00<?, ?it/s]

Epoch: 1
Train Instance Accuracy: 0.171168
Train Loss: 5811.680797


  0%|          | 0/2962 [00:00<?, ?it/s]

Epoch: 2
Train Instance Accuracy: 0.171506
Train Loss: 5389.464313


  0%|          | 0/2962 [00:00<?, ?it/s]

...

Epoch: 98
Train Instance Accuracy: 0.164078
Train Loss: 5308.843948


  0%|          | 0/2962 [00:00<?, ?it/s]

Epoch: 99
Train Instance Accuracy: 0.163403
Train Loss: 5309.073992


  0%|          | 0/2962 [00:00<?, ?it/s]

Epoch: 100
Train Instance Accuracy: 0.167286
Train Loss: 5308.903368


1.7984883785247803

In [22]:
mean_correct

[0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.5,
 0.5,
 0.5,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.5,
 0.0,
 0.5,
 0.0,
 0.5,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.5,
 0.0,
 0.5,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.5,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.5,
 0.0,
 0.5,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.5,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.5,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.5,
 0.5,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.5,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5,
 0.5

In [21]:
print(xyz.shape, norm.shape)

torch.Size([1, 3, 25000]) torch.Size([1, 3, 25000])


In [9]:
print(points.shape, target.shape)
points = points.transpose(2, 1)
norm = points[:, 3:, :]
xyz = points[:, :3, :]
print(xyz.shape, norm.shape)

torch.Size([2, 25000, 6]) torch.Size([2, 6])


tensor([[[-0.6256, -0.6436, -0.6450,  ..., -0.2290, -0.2155, -0.0489],
         [-0.0010,  0.0036,  0.0183,  ...,  0.4033,  0.3900, -0.4553],
         [ 0.5096,  0.5026,  0.4961,  ...,  0.0542,  0.0459,  0.4762],
         [-0.5164, -0.1890, -0.1587,  ..., -0.4889, -0.6450,  0.3898],
         [ 0.2294,  0.3115,  0.3833,  ..., -0.4809, -0.3864, -0.1763],
         [ 0.8201,  0.9073,  0.8844,  ..., -0.6846, -0.5391,  0.8114]],

        [[ 0.2154,  0.2293,  0.2213,  ..., -0.2250, -0.2201, -0.2116],
         [-0.4338, -0.4366, -0.4114,  ..., -0.1138, -0.0995, -0.0837],
         [ 0.1508,  0.1459,  0.1479,  ..., -0.4035, -0.4049, -0.4049],
         [ 0.2925,  0.4119,  0.5507,  ..., -0.4522, -0.4908, -0.6138],
         [-0.0279,  0.1556, -0.0437,  ...,  0.2790,  0.1993,  0.0470],
         [ 0.9413,  0.8884,  0.8224,  ...,  0.8279,  0.8288,  0.7716]]])

In [9]:
classifier(xyz, norm)

RuntimeError: falseINTERNAL ASSERT FAILED at "/lambda_stor/homes/songhao.jiang/projects/point_cloud/pkgs/Pointnet2_PyTorch/pointnet2_ops_lib/pointnet2_ops/_ext-src/src/sampling.cpp":83, please report a bug to PyTorch. CPU not supported